In [1]:
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
# loss functionの初期化、定義
loss_fn = nn.CrossEntropyLoss()

In [5]:
#optimizer 最適化器
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        #予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)

        #バックプロパゲーション 誤差逆伝播法
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done")

Epoch 1
-------------------------------
loss: 2.302560 [    0/60000]
loss: 2.276917 [ 6400/60000]
loss: 2.212439 [12800/60000]
loss: 2.196878 [19200/60000]
loss: 2.167969 [25600/60000]
loss: 2.064299 [32000/60000]
loss: 2.102290 [38400/60000]
loss: 2.012712 [44800/60000]
loss: 2.014941 [51200/60000]
loss: 1.935336 [57600/60000]
Test Error: 
 Accuracy: 48.3%, Avg loss: 0.031167 

Epoch 2
-------------------------------
loss: 2.003494 [    0/60000]
loss: 2.031614 [ 6400/60000]
loss: 1.888268 [12800/60000]
loss: 1.916618 [19200/60000]
loss: 1.877655 [25600/60000]
loss: 1.735315 [32000/60000]
loss: 1.817024 [38400/60000]
loss: 1.710289 [44800/60000]
loss: 1.750993 [51200/60000]
loss: 1.637340 [57600/60000]
Test Error: 
 Accuracy: 49.2%, Avg loss: 0.027447 

Epoch 3
-------------------------------
loss: 1.779155 [    0/60000]
loss: 1.851311 [ 6400/60000]
loss: 1.664570 [12800/60000]
loss: 1.718198 [19200/60000]
loss: 1.684581 [25600/60000]
loss: 1.525961 [32000/60000]
loss: 1.611408 [38400/